# 主流币种的波段资金追踪

1. 根据币种选择波段，使用Chainbase平台对事件段内的账号收益进行排序,

以下是一段UNI的例子
```sql
With
    prices as (
        SELECT
            toStartOfMinute(timestamp) as time,
            price
        FROM
            prices.token_prices
        WHERE
            contract_address = '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984'
            AND timestamp >= '2022-09-01'
    ),
    start_price as (
        select
            price
        from
            prices
        order by
            time
        limit
            1
    ), end_price as (
        select
            price
        from
            prices
        order by
            time desc
        limit
            1
    ), transfers as (
        SELECT
            toStartOfMinute(block_timestamp) AS time,
            contract_address,
            from_address,
            to_address,
            (CAST(value AS Float64) / 1000000000000000000) AS amount
        FROM
            ethereum.token_transfers
        WHERE
            ethereum.token_transfers.contract_address = '0x1f9840a85d5af5bf1d1762f925bdaddc4201f984'
            AND ethereum.token_transfers.block_timestamp > '2022-09-01'
    ),
    buys as (
        select
            to_address as address,
            sum(amount) as total_amount,
            sum(amount * prices.price) as cost,
            cost / total_amount as buy_avg_price
        from
            transfers
            left join prices on transfers.time = prices.time
        group by
            to_address
    ),
    sells as (
        select
            from_address as address,
            sum(amount) as total_amount,
            sum(amount * prices.price) as cost,
            cost / total_amount as sell_avg_price
        from
            transfers
            left join prices on transfers.time = prices.time
        group by
            from_address
    )
select
    buys.address,
    buys.total_amount as in_amount,
    COALESCE(
        buys.buy_avg_price,
        (
            select
                price
            from
                start_price
        )
    ) as in_price,
    sells.total_amount as out_amount,
    COALESCE(
        sells.sell_avg_price,
        (
            select
                price
            from
                end_price
        )
    ) as out_price,
    out_price - in_price as delta_price
from
    buys OUTER
    JOIN sells on buys.address = sells.address
where
    in_amount > 1000
    or out_amount > 1000
order by
    delta_price DESC
```